In [1]:
import torch

In [2]:
! pip install segmentation_models_pytorch

     |████████████████████████████████| 42 kB 228 kB/s eta 0:00:011
     |████████████████████████████████| 58 kB 1.5 MB/s eta 0:00:011
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=1bd8c9b979338e9223284cc3f723433b9f4321547f5187203921c9eaae829b31
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16035 sha256=062133c28c605c7ab8445c457fe963f1a9757651f729adaa8c82d4445a46d5a8
  Stored in directory: /root/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
Successfully built pretrainedmodels efficientnet-pytorch
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


# MAKE SUBMIT.CSV

In [5]:
import pandas as pd
import torch
import os
import cv2
import segmentation_models_pytorch as smp
from tqdm import tqdm_notebook as tqdm

def test_generator(transforms, path_to_images='test_images'):
    images_name = os.listdir(path_to_images)

    for name in images_name:
        image = cv2.imread(os.path.join(path_to_images, name))
        image = transforms(image=image)['image']
        yield image, name


def make_submission(
        model,
        path_to_images='test_images',
        device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
        threshold=0.5, min_size=[500, 500, 500, 500]):

    model.to(device)
    model.eval()
    transforms = get_transforms(phase='val')
    test_images = test_generator(transforms, path_to_images)
    result = pd.DataFrame(columns=['ImageId', 'EncodedPixels', 'ClassId'])
    pbar = tqdm(test_images, total=5506)
    for image, name in test_images:
        output = model(image.unsqueeze(0).to(device)).cpu().detach().squeeze()
        pred = predict(output, threshold=threshold, min_size=min_size)
        for defect in range(4):
            rle = mask_to_rle(pred[defect])
            result = result.append({'ImageId': name, 'EncodedPixels': rle, 'ClassId': defect + 1}, ignore_index=True)
        pbar.update(1)
    pbar.close()
    return result

if __name__ == '__main__':
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = smp.Unet('resnet34', encoder_weights=None, classes=4, activation=None)
    state = torch.load('../input/best-model/model_epoch_14_score_0.9430.pth', map_location=device)
    model.load_state_dict(state['state_dict'])
    for i in [0.5, 0.6, 0.7, 0.8]:
        result = make_submission(model, threshold=i, path_to_images='../input/severstal-steel-defect-detection/test_images')
        result['ImageId_ClassId'] = result['ImageId'] + '_' + result['ClassId'].apply(str)
        result[['EncodedPixels','ImageId_ClassId']].to_csv(str(i)+'submission.csv', index=False)
    
    result.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
result[['EncodedPixels','ImageId_ClassId']].to_csv('submission1.csv', index=False)

# UTILS

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import torch

DATA_DIR = '../input/severstal-steel-defect-detection/train_images'
def mask_to_rle(img):
    '''
    img: numpy array, 1 -> mask, 0 -> background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle_to_mask(rle, shape=(1600, 256)):
    """
    :param rle: run-length as string formated (start length)
    :param shape: (width,height) of array to return
    :return: numpy array, 1 - mask, 0 - background
    """
    runs = np.array([int(x) for x in rle.split()])
    runs[1::2] += runs[::2]
    runs -= 1
    starts, ends = runs[::2], runs[1::2]
    mask = np.zeros(shape[0] * shape[1])
    for start, end in zip(starts, ends):
        mask[start:end] = 1
    return mask.reshape(shape).T


def show_defects(image, mask, pallet=((249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12))):

    for i in range(4):
        image[0, mask[i] == 1] = 255
    plt.imshow(image.permute(1, 2, 0))
    plt.show()


def show_mask_image(image, mask, pallet=((249, 192, 12), (0, 185, 241), (114, 0, 218), (249, 50, 12))):
    fig, ax = plt.subplots(figsize=(15, 15))
    image = image.permute(1, 2, 0).numpy()
    mask = mask.permute(1, 2, 0)

    for ch in range(4):
        image[mask[:, :, ch] == 1] = pallet[ch]
    plt.imshow(image)

    plt.show()


def make_mask(name, df):
    mask = np.zeros((256, 1600, 4), dtype=np.float32)
    rows = df.loc[name]
    for defect in range(1, 5):
        rle = rows[defect]
        if not pd.isna(rle):
            encoded = rle_to_mask(rle)
            mask[:, :, defect - 1] = encoded
    return mask

def dice_single_channel(targets, preds, eps=1e-9):
    batch_size = preds.shape[0]
    preds = preds.view((batch_size, -1)).float()
    targets = targets.view((batch_size, -1)).float()
    dice = (2 * (preds * targets).sum(1) + eps) / (preds.sum(1) + targets.sum(1) + eps)
    return dice


def mean_dice_score(targets, outputs, threshold=0.5, min_size=[600, 600, 1000, 2000]):
    batch_size = outputs.shape[0]
    n_channels = outputs.shape[1]
    
    preds = torch.zeros_like(outputs)
    for i in range(batch_size):
        preds[i] = predict(output=outputs[i], threshold=threshold, min_size=min_size)

    mean_dice = 0
    for i in range(n_channels):
        dice = dice_single_channel(targets[:, i, :, :], preds[:, i, :, :])
        mean_dice += dice.sum(0) / (n_channels * batch_size)
    return mean_dice.item()


def pixel_accuracy_score(targets, outputs, threshold=0.5, min_size=[600, 600, 1000, 2000]):
    preds = torch.zeros_like(outputs)
    for i in range(outputs.shape[0]):
        preds[i] = predict(output=outputs[i], threshold=threshold, min_size=min_size)
    correct = torch.sum((targets == preds)).item()
    total = outputs.numel()
    return correct / total


def epoch_metrics(targets, outputs, threshold=0.5):
    return {'dice': mean_dice_score(targets, outputs, threshold),
            'pixel_acc': pixel_accuracy_score(targets, outputs, threshold)}


def predict(output, threshold=0.5, min_size=[600, 600, 1000, 2000]):
    output = output.sigmoid()
    prediction = torch.zeros_like(output)
    for i in range(4):
        pixels = (output[i] > threshold).float().sum()
        if pixels < min_size[i]:
            prediction[i] = torch.zeros_like(output[i])
        else:
            prediction[i] = (output[i] > threshold).float()
    return prediction

from torch.utils.data import DataLoader, Dataset
import pandas as pd
import cv2
import os
import albumentations as albu
import albumentations.pytorch as albu_pytorch
from sklearn.model_selection import train_test_split


class SteelDataset(Dataset):
    def __init__(self, dataset, phase='train', data_dir=DATA_DIR, image_size=(256, 1600), n_classes=4):
        self.dataset = dataset
        self.phase = phase
        self.dir = data_dir
        self.transforms = get_transforms(phase=self.phase)
        self.image_size = image_size
        self.n_classes = n_classes

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        name = self.dataset.iloc[index].name
        image = cv2.imread(os.path.join(self.dir, name))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = make_mask(name, self.dataset)

        transformed = self.transforms(image=image, mask=mask)
        image, mask = transformed['image'], transformed['mask'].permute(2, 0 , 1)

        return image, mask


def get_transforms(list_transforms=None, phase='train'):
    if not list_transforms:
        list_transforms = []

    if phase == 'train':
        list_transforms.extend(
            [
                albu.RandomBrightnessContrast(p=0.1, brightness_limit=0.1, contrast_limit=0.1),
                albu.HorizontalFlip(p=0.5),
                albu.VerticalFlip(p=0.5),
                # albu.ElasticTransform(p=0.5),
                # albu.GridDistortion(p=0.5),
                # albu.OpticalDistortion(p=0.5),
            ]
        )
    list_transforms.extend(
        [
            albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            albu_pytorch.ToTensorV2()
        ]
    )

    list_transforms = albu.Compose(list_transforms)
    return list_transforms


def data_provider(df, batch_size=8, shuffle=True, stratify_by=None):

    if stratify_by:
        train_df, val_df = train_test_split(df, test_size=0.2,
                                            stratify=df[stratify_by],
                                            random_state=42,
                                            shuffle=shuffle)
    else:
        train_df, val_df = train_test_split(df, test_size=0.2,
                                            random_state=42,
                                            shuffle=shuffle)

    dataloader = {'train': DataLoader(SteelDataset(train_df, phase='train'), batch_size=batch_size),
                  'val': DataLoader(SteelDataset(val_df, phase='val'), batch_size=batch_size)}


    return dataloader


 # VALIDATION


In [6]:
import json
import torch
from torch import nn
import segmentation_models_pytorch as smp
import numpy as np
import pandas as pd
import time
from tqdm import tqdm_notebook as tqdm


class Trainer:
    def __init__(self, model, device, data_frame, batch_size=8, stratify_by=None):
        self.model = model
        self.device = device
        self.model = model.to(device)
        
        self.device = device
        self.batch_size = batch_size
        
        self.df = data_frame
        self.dataloaders = data_provider(self.df, batch_size=self.batch_size, stratify_by=stratify_by)
        
        self.metrics = {'dice': mean_dice_score, 'pixel_acc': pixel_accuracy_score}
        self.metrics_values = {phase: {name: [] for name in self.metrics.keys()}
                               for phase in ['train', 'val']}
        
        self.best_score =np.array([-np.inf for _ in self.metrics.keys()])

        
    def step(self, threshold, min_size, phase='val'):

        metrics = {name: [] for name in self.metrics.keys()}
        epoch_metric = {}

        self.model.eval()

        dataloader = self.dataloaders[phase]
        pbar = tqdm(dataloader, total=len(dataloader))

        for i, (images, targets) in enumerate(dataloader):
            images, targets = images.to(self.device), targets.to(self.device)

            with torch.set_grad_enabled(phase == 'train'):
                outputs = self.model(images)
        
                for metric in self.metrics.keys():
                    metrics[metric].append(self.metrics[metric](targets, outputs, threshold, min_size))

                pbar.update(1)
                
        pbar.close()
        
        for metric in self.metrics.keys():
            epoch_metric[metric] = np.mean(metrics[metric])
            self.metrics_values[phase][metric].append(epoch_metric[metric])

        del images, targets, outputs
        torch.cuda.empty_cache()
        return epoch_metric

    def train(self):
        for threshold in tqdm(np.arange(0.3, 0.46, 0.01)):
        
            metric = self.step('val')
            print('|val_metric {}'.format(metric))
            
            scores = np.fromiter(metric.values(), dtype=np.float)
            if (scores[0] > self.best_score[0]).all():
                print('-' * 10 + 'New optimal threshold found and saved' + '-' * 10)
                self.best_score = scores


if __name__ == '__main__':
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = smp.Unet('resnet34', encoder_weights=None, classes=4, activation=None)
    state = torch.load('../input/best-model/model_epoch_14_score_0.9430.pth', map_location=device)
    model.load_state_dict(state['state_dict'])
    
    df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
    without_def = set(os.listdir('../input/severstal-steel-defect-detection/train_images')) - set(list(df['ImageId']))
    df = df.pivot(index='ImageId', columns='ClassId', values='EncodedPixels')
    for i in list(without_def):
        df.loc[i] = [pd.NA, pd.NA, pd.NA, pd.NA]
        
    df['NumDefects'] = df.count(axis=1)
    
    model_train = Trainer(
        model=model,
        device=device,
        batch_size=12,
        data_frame=df,
        stratify_by='NumDefects'
    )
    

    min_size = [600, 600, 1000, 2000]

FileNotFoundError: [Errno 2] No such file or directory: 'train_images'

In [ ]:
best = 0.8903
for i in np.arange(0.7, 0.9, 0.01):
    for j in [[500, 500, 500, 500], [600, 600, 1000, 2000]]:
        print(i, j)
        metric = model_train.step(threshold=i, min_size=j)
        if metric['dice'] > best:
            best = metric['dice']
            best_param = (i, j)
        print(metric)

In [ ]:
best_param

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
import os
import cv2
import segmentation_models_pytorch as smp

class ModelCombinet(nn.Module):
    def __init__(self, models):
        super(ModelCombinet, self).__init__()
        self.models = models
    
    def __call__(self, x):
        res = []
        x = x.cuda()
        with torch.no_grad():
            for m in self.models:
                res.append(m(x))
        res = torch.stack(res)
        return torch.mean(res, dim=0)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model1 = smp.Unet('resnet34', encoder_weights=None, classes=4, activation=None)
state1 = torch.load('../input/best-model/model_epoch_1_score_0.8776.pth', map_location=device)
model1.load_state_dict(state1['state_dict'])
model1.to(device)

model2 = smp.Unet('resnet34', encoder_weights=None, classes=4, activation=None)
state2 = torch.load('../input/best-model/model_epoch_21_score_0.8714.pth', map_location=device)
model2.load_state_dict(state2['state_dict'])
model2.to(device)    

model3 = smp.Unet('resnet18', encoder_weights=None, classes=4, activation=None)
state3 = torch.load('../input/best-model/model_epoch_19_score_0.8847 (1).pth', map_location=device)
model3.load_state_dict(state3['state_dict'])
model3.to(device)

df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
df = df.pivot(index='ImageId', columns='ClassId', values='EncodedPixels')
df['NumDefects'] = df.count(axis=1)
    
    
model = ModelCombinet([model1, model2, model3])
# model_train = Trainer(
#         model=model,
#         device=device,
#         batch_size=12,
#         data_frame=df,
#         stratify_by='NumDefects'
# )

In [ ]:
model_train.step(threshold=0.38, min_size=[600, 600, 1000, 2000])

In [ ]:
{'dice': 0.8869642785617283, 'pixel_acc': 0.9940975052969796}
{'dice': 0.8885742255619594, 'pixel_acc': 0.9940522198449998}